In [1]:
!pip install tensorflow==2.9.1 tensorflow-gpu==2.9.1 opencv-python matplotlib

  Using cached tensorflow-2.9.1-cp310-cp310-win_amd64.whl (444.1 MB)


In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2
import math

In [2]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()

In [46]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    #[0:16] = [nose, left eye, right eye, left ear, 
    #right ear, left shoulder, right shoulder, 
    #left elbow, right elbow, left wrist, 
    #right wrist, left hip, right hip, 
    #left knee, right knee, left ankle, right ankle]
    
    #for kp in shaped:
    #    ky, kx, kp_conf = kp
    #    if kp_conf > confidence_threshold:
    #        cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)
    
    cv2.circle(frame, (int(shaped[3][1]), int(shaped[3][0])), 7, (0,0,255), -1) #Left ear, Red
    cv2.circle(frame, (int(shaped[4][1]), int(shaped[4][0])), 7, (255,0,0), -1) #Right ear, Blue

In [79]:
def face_angle(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    #middle_eartoear(x,y) = (middle_eartoear[1],middle_eartoear[0])
    #nose(x,y) = (shaped[0][1],shaped[0][0])
    middle_eartoear = (shaped[4] + shaped[3])/2
    vector_len = math.dist(middle_eartoear[:2],shaped[0][:2])
    temp_x = (shaped[0][1]-middle_eartoear[1])/vector_len
    temp_y = (shaped[0][0]-middle_eartoear[0])/vector_len
    face_angle = math.degrees(math.acos(temp_x))
    
    if shaped[0][0] > middle_eartoear[0]:
        face_angle = -face_angle+360
    
    if shaped[3][1] < shaped[4][1]: #หน้าหลัง
        back_or_front = 'Back'
    elif shaped[3][1] >= shaped[4][1]:
        back_or_front = 'Front'

    vector_len_threshold = 20
    if vector_len < vector_len_threshold:
        look_udlr = ' '
    elif vector_len >= vector_len_threshold:
        if (0 < face_angle < 45) or (315 < face_angle < 360):
            look_udlr = 'Right'
        elif (45 < face_angle < 135):
            look_udlr = 'Up'
        elif (135 < face_angle < 225):
            look_udlr = 'Left'
        elif (225 < face_angle < 315):
            look_udlr = 'Down'
    
    if (shaped[4][2] > confidence_threshold) & (shaped[3][2] > confidence_threshold) & (shaped[0][2] > confidence_threshold):
        cv2.putText(frame,back_or_front+'('+str(int(vector_len))+', '+str(int(face_angle))+')', (int(middle_eartoear[1]+5),int(middle_eartoear[0]+15)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0),2)
        cv2.putText(frame,back_or_front+' '+look_udlr, (int(middle_eartoear[1]),int(middle_eartoear[0])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2)

In [38]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))

#    for edge, color in edges.items():
#        p1, p2 = edge
#        y1, x1, c1 = shaped[p1]
#        y2, x2, c2 = shaped[p2]
        
#        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
#            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

    middle_eartoear = (shaped[4] + shaped[3])/2
    
    if (shaped[4][2] > confidence_threshold) & (shaped[3][2] > confidence_threshold) & (shaped[0][2] > confidence_threshold):
        cv2.arrowedLine(frame, (int(middle_eartoear[1]), int(middle_eartoear[0])), (int(shaped[0][1]), int(shaped[0][0])), (0,0,255), 2)

In [80]:
cap = cv2.VideoCapture(0) # 'test001.mp4'

while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions  
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    #draw_keypoints(frame, keypoints_with_scores, 0.4)
    face_angle(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break

    if ret == False:
        break
        
cap.release()
cv2.destroyAllWindows()